In [4]:
# Inspiration on reading in Chunks from: https://pythonspeed.com/articles/chunking-pandas/
# and https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c

import pandas as pd

FILE_PATH = 'in/versions-1.6.0-2020-01-12.csv'

df_chunk = pd.read_csv(FILE_PATH, chunksize=100000) #nrows=2

chunk_list = [] 


def _apply_filer(df, target):
    df = df[['Platform','Project Name', 'Number']]
    return df[df['Platform'] == target]

def chunks_df(platform):
    for chunk in df_chunk:  
        chunk_filter = _apply_filer(chunk, platform)
        chunk_list.append(chunk_filter)
        
    df_concat = pd.concat(chunk_list)
    df_concat.to_csv(f'out/ver-libio-{platform}.csv', index=False)

chunks_df('Cargo')


In [7]:
import pandas as pd

ECOSYS = 'Cargo'
print(f"Checking {ECOSYS}")

def _get_frames(target):
    ltarget = target.lower()
    df = pd.read_csv(f"in/{ltarget}_versions_05-17-2022.csv")
    df_libio = pd.read_csv(f'out/ver-libio-{target}.csv')
    return df, df_libio

df, df_libio = _get_frames(ECOSYS) #Enter case sensitive Libraries io naming
print(f"Total versions DaSEA : {len(df)}, Libraries io: {len(df_libio)}")

# Get packages that we share to only look at those versions
def _get_shared_packages_sample(target):
    packages_df = pd.read_csv(f'in/{target}_packages_05-17-2022.csv')
    packages_libio_df = pd.read_csv('in/projects-1.6.0-2020-01-12.csv')
    names_dasea = packages_df['name'].str.lower()
    names_libio = packages_libio_df[packages_libio_df['Platform'] == f'{target}']['Name'].str.lower()
    res = names_libio[names_libio.isin(names_dasea)]
    print(f"DaSEA and {target} shares {len(res)}") # Cargo 3.4660 = 35635 - 975 (Correct)
    return res

shared_packages = _get_shared_packages_sample(ECOSYS)


Checking Cargo
Total versions DaSEA : 543138, Libraries io: 195011


<ipython-input-7-3d7b3dbda04c>:25: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  shared_packages = _get_shared_packages_sample(ECOSYS)


DaSEA and Cargo shares 34660


4235788                    fontdue
3544907    oasis-game-client-proxy
3560446                       wood
4598945                    sql-ast
124314                 d3d10_1-sys
3029762                      widow
1221349               simple-error
2053889                      munch
3473332               snafu-derive
1562493                      vikos
2331902                   tss-sapi
4304724        sky-color-wallpaper
2006851               websocket-vi
703196                 sodiumoxide
730195                    dispatch
2587214                 membarrier
1760417              string-intern
2237055                     extfsm
4284838                  dune-full
4572429                      rfm69
Name: Name, dtype: object

In [62]:
# Get one sample package of the packcages shared
sample = shared_packages.sample(1)
sample

2987554    ene-core
Name: Name, dtype: object

In [63]:
# Get versions for those packages in Libraries
ver_libio = df_libio[df_libio['Project Name'].isin(sample)]
ver_libio.to_csv('out/ver_sample_libio.csv', index=False)

# Get a list of the unique packages that sample package is dependent on
uniq_libio = ver_libio['Project Name'].unique()

# For each unique package, get versions
for e in uniq_libio:
    print(f"Package: {e}")
    v_df = ver_libio[ver_libio['Project Name'] == f'{e}'].sort_values('Number')
    v_lst = v_df['Number'].to_list()
    print(f"Libraries: {v_lst}")
    dasea_v_df = df[df["name"] == f'{e}']['version'].sort_values().to_list()
    print(f"DaSEA: {dasea_v_df}")
    print()
    print("Conclusion:")




Package: ene-core
Libraries: ['0.0.0']
DaSEA: ['0.0.0']

Conclusion:
